In [1]:
import requests as rq
from requests.structures import CaseInsensitiveDict

In [8]:
host = "http://128.179.186.140"

# 1. a_create-credential-schema

In [162]:
headers = CaseInsensitiveDict()
headers["Content-Type"] = "application/json"
headers["accept"] = "application/json"

data = """
{
  "attributes": [
    "score"
  ],
  "schema_name": "score_schema_2",
  "schema_version": "1.0"
}
"""

res = rq.post(f'{host}:9020/schemas?create_transaction_for_endorser=false', headers=headers, data=data)
res.text

'{"sent": {"schema_id": "GxgDCXeCGmqqz259Mva9h6:2:score_schema_2:1.0", "schema": {"ver": "1.0", "id": "GxgDCXeCGmqqz259Mva9h6:2:score_schema_2:1.0", "name": "score_schema_2", "version": "1.0", "attrNames": ["score"], "seqNo": 63845}}, "schema_id": "GxgDCXeCGmqqz259Mva9h6:2:score_schema_2:1.0", "schema": {"ver": "1.0", "id": "GxgDCXeCGmqqz259Mva9h6:2:score_schema_2:1.0", "name": "score_schema_2", "version": "1.0", "attrNames": ["score"], "seqNo": 63845}}'

In [256]:
#schema = res.json()['sent']['schema_id']
schema = "GxgDCXeCGmqqz259Mva9h6:2:address_schema:1.0"

In [255]:
rq.get(f'{host}:9020/schemas/created').json()

{'schema_ids': ['GxgDCXeCGmqqz259Mva9h6:2:bruno_test:1.0',
  'GxgDCXeCGmqqz259Mva9h6:2:my_schema:1.0',
  'GxgDCXeCGmqqz259Mva9h6:2:my_schemav1.0:1.0',
  'GxgDCXeCGmqqz259Mva9h6:2:oisin_schema1:1.0',
  'GxgDCXeCGmqqz259Mva9h6:2:score_schema_2:1.0',
  'GxgDCXeCGmqqz259Mva9h6:2:FSFilet:1.0',
  'GxgDCXeCGmqqz259Mva9h6:2:prefs:1.0',
  'GxgDCXeCGmqqz259Mva9h6:2:address_schema:1.0']}

# 1. b_define-credential-without-revocation

In [257]:
headers = CaseInsensitiveDict()
headers["Content-Type"] = "application/json"
headers["accept"] = "application/json"

data = """
{
  "revocation_registry_size": 1000,
  "schema_id": "%s",
  "support_revocation": true,
  "tag": "RevokableAddressTwo"
}
"""%schema

res1 = rq.post(f'{host}:9020/credential-definitions?create_transaction_for_endorser=false', headers=headers, data=data)
res1.text

'{"sent": {"credential_definition_id": "GxgDCXeCGmqqz259Mva9h6:3:CL:63836:RevokableAddressTwo"}, "credential_definition_id": "GxgDCXeCGmqqz259Mva9h6:3:CL:63836:RevokableAddressTwo"}'

In [258]:
rq.get(f"{host}:9020/credential-definitions/created").json()

{'credential_definition_ids': ['GxgDCXeCGmqqz259Mva9h6:3:CL:63845:ScoreAttemptNPlusOne',
  'GxgDCXeCGmqqz259Mva9h6:3:CL:63795:bruno_test',
  'GxgDCXeCGmqqz259Mva9h6:3:CL:63797:Github',
  'GxgDCXeCGmqqz259Mva9h6:3:CL:63797:LauzhackDemo',
  'GxgDCXeCGmqqz259Mva9h6:3:CL:63845:RevokableAddress',
  'GxgDCXeCGmqqz259Mva9h6:3:CL:63836:RevokableAddress',
  'GxgDCXeCGmqqz259Mva9h6:3:CL:63796:LauzhackDemoRevoke2',
  'GxgDCXeCGmqqz259Mva9h6:3:CL:63796:LauzhackDemoRevoke',
  'GxgDCXeCGmqqz259Mva9h6:3:CL:63796:LauzhackDemo',
  'GxgDCXeCGmqqz259Mva9h6:3:CL:63836:RevokableAddressTwo',
  'GxgDCXeCGmqqz259Mva9h6:3:CL:63796:LauzhackDemoRevoke3']}

In [259]:
res1.json()

{'sent': {'credential_definition_id': 'GxgDCXeCGmqqz259Mva9h6:3:CL:63836:RevokableAddressTwo'},
 'credential_definition_id': 'GxgDCXeCGmqqz259Mva9h6:3:CL:63836:RevokableAddressTwo'}

In [260]:
#cred_def = res1.json()['credential_definition_id']
cred_def = "GxgDCXeCGmqqz259Mva9h6:3:CL:63836:RevokableAddressTwo"
cred_def

'GxgDCXeCGmqqz259Mva9h6:3:CL:63836:RevokableAddressTwo'

# 2. a_create-a-connection

In [240]:
headers = CaseInsensitiveDict()
headers["accept"] = "application/json"

data = """
''
"""

res2 = rq.post(f'{host}:8080/connection/invitation', headers=headers, data=data)
res2.text

'{"invitationUrl":"http://1763-2001-620-618-5b8-2-80b3-0-790.eu.ngrok.io?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiZWUwMWQ2MDItYjQzNC00M2JhLTgwMjktYzE5MmM2NmRiNjU5IiwgInJlY2lwaWVudEtleXMiOiBbIjdwdmtBdzVzektMVzNwa3RERjVFR1hUaDZ6UXpIY0xQUUFQQWtxZWJ3cVlTIl0sICJsYWJlbCI6ICJBZG5vdnVtIFZDTVMgRGV2IiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vMTc2My0yMDAxLTYyMC02MTgtNWI4LTItODBiMy0wLTc5MC5ldS5uZ3Jvay5pbyJ9","connectionId":"65ef9fa9-b278-41ee-a58b-b4b79da7099a"}'

In [241]:
res2.json()['connectionId']

'65ef9fa9-b278-41ee-a58b-b4b79da7099a'

# 2. b_verify-connection-status

In [242]:
headers = CaseInsensitiveDict()
headers["accept"] = "application/json"

connectionid = res2.json()['connectionId']
print(connectionid)

res3 = rq.get(f'{host}:8080/connection/{connectionid}', headers=headers)
res3.text

65ef9fa9-b278-41ee-a58b-b4b79da7099a


'"responded"'

# 3. a_issue-a-credential

In [243]:
print(cred_def)

GxgDCXeCGmqqz259Mva9h6:3:CL:63845:ScoreAttemptNPlusOne


In [244]:
headers = CaseInsensitiveDict()
headers["accept"] = "application/json"
headers["Content-Type"] = "application/json"

data = """
{"connectionId":"%s",
"credentialDefinitionId":"%s",
"attributes": {"score": "45"}
}'
"""% (res2.json()['connectionId'], cred_def)

res4 = rq.post(f'{host}:8100/issue/process', headers=headers, data=data)
res4.text

'{"status":"IN_PROGRESS","processId":"1f807646-1e8e-44df-b0e9-cb519e35e6b1"}'

In [245]:
process_id = res4.json()['processId']
process_id

'1f807646-1e8e-44df-b0e9-cb519e35e6b1'

# 3 b_check-issuing-state

In [248]:
headers = CaseInsensitiveDict()
headers["accept"] = "application/json"

connectionid = res2.json()['connectionId']
print(connectionid)

res5 = rq.get(f'{host}:8100/issue/process/{process_id}/state', headers=headers)
res5.text

65ef9fa9-b278-41ee-a58b-b4b79da7099a


'"VC_ISSUED"'

# 4 a_verify-credential

In [249]:
headers = CaseInsensitiveDict()
headers["accept"] = "application/json"
headers["Content-Type"] = "application/json"

data = """
{"connectionId":"%s",
"credentialDefinitionId":"%s",
"attributes": ["score"]
}'
"""% (res2.json()['connectionId'], cred_def)

res6 = rq.post(f'{host}:8081/verify/process', headers=headers, data=data)
res6.text

'{"status":"IN_PROGRESS","processId":"893422fb-1ac7-4111-925e-cab0e9388160"}'

In [250]:
newprocess_id = res6.json()['processId']
newprocess_id

'893422fb-1ac7-4111-925e-cab0e9388160'

# 4 b_check-verification-state

In [252]:
headers = CaseInsensitiveDict()
headers["accept"] = "application/json"

res7 = rq.get(f'{host}:8081/verify/process/{newprocess_id}/state', headers=headers)
res7.text

'"VERIFIED"'

# 4 c_get-claims

In [253]:
headers = CaseInsensitiveDict()
headers["accept"] = "application/json"

res8 = rq.get(f'{host}:8081/verify/process/{newprocess_id}/claims', headers=headers)
res8.text

'{"score":"45"}'

# 5 get_registries

In [163]:
headers = CaseInsensitiveDict()
headers["accept"] = "application/json"

res9 = rq.get(f'{host}:9020/revocation/registries/created', headers=headers)
res9.text

'{"rev_reg_ids": ["GxgDCXeCGmqqz259Mva9h6:4:GxgDCXeCGmqqz259Mva9h6:3:CL:63796:LauzhackDemoRevoke2:CL_ACCUM:72f71eda-e787-4536-8f91-72560ae1f168", "GxgDCXeCGmqqz259Mva9h6:4:GxgDCXeCGmqqz259Mva9h6:3:CL:63796:LauzhackDemoRevoke2:CL_ACCUM:1ab68c5a-0140-4031-9f74-cb4dd93c18a0", "GxgDCXeCGmqqz259Mva9h6:4:GxgDCXeCGmqqz259Mva9h6:3:CL:63796:LauzhackDemoRevoke3:CL_ACCUM:d94b084d-1ce3-4eaa-b7f7-ab60bdca9c37", "GxgDCXeCGmqqz259Mva9h6:4:GxgDCXeCGmqqz259Mva9h6:3:CL:63796:LauzhackDemoRevoke:CL_ACCUM:b3eca0a6-f2c3-4836-9766-7033ae64df0e", "GxgDCXeCGmqqz259Mva9h6:4:GxgDCXeCGmqqz259Mva9h6:3:CL:63796:LauzhackDemoRevoke3:CL_ACCUM:d641ffcc-2abf-4627-a04a-50b9e4b281f8"]}'

In [167]:
headers = CaseInsensitiveDict()
headers["accept"] = "application/json"

res10 = rq.get(f'{host}:9020/revocation/active-registry/{cred_def}', headers=headers)
res10.text

'{"result": {"cred_def_id": "GxgDCXeCGmqqz259Mva9h6:3:CL:63796:LauzhackDemoRevoke3", "tag": "d641ffcc-2abf-4627-a04a-50b9e4b281f8", "tails_public_uri": "http://128.179.184.61:6543/GxgDCXeCGmqqz259Mva9h6:4:GxgDCXeCGmqqz259Mva9h6:3:CL:63796:LauzhackDemoRevoke3:CL_ACCUM:d641ffcc-2abf-4627-a04a-50b9e4b281f8", "record_id": "d641ffcc-2abf-4627-a04a-50b9e4b281f8", "state": "active", "tails_hash": "GXaF48RhPFvVhQDhi2rTQDbQzXeQykNn6iNCGwcbh68V", "tails_local_path": "/home/indy/.indy_client/tails/GxgDCXeCGmqqz259Mva9h6:4:GxgDCXeCGmqqz259Mva9h6:3:CL:63796:LauzhackDemoRevoke3:CL_ACCUM:d641ffcc-2abf-4627-a04a-50b9e4b281f8/GXaF48RhPFvVhQDhi2rTQDbQzXeQykNn6iNCGwcbh68V", "revoc_reg_id": "GxgDCXeCGmqqz259Mva9h6:4:GxgDCXeCGmqqz259Mva9h6:3:CL:63796:LauzhackDemoRevoke3:CL_ACCUM:d641ffcc-2abf-4627-a04a-50b9e4b281f8", "revoc_def_type": "CL_ACCUM", "issuer_did": "GxgDCXeCGmqqz259Mva9h6", "revoc_reg_def": {"ver": "1.0", "id": "GxgDCXeCGmqqz259Mva9h6:4:GxgDCXeCGmqqz259Mva9h6:3:CL:63796:LauzhackDemoRevoke3:CL_A

In [168]:
revoc_reg_id = res10.json()['result']['revoc_reg_id']
revoc_reg_id

'GxgDCXeCGmqqz259Mva9h6:4:GxgDCXeCGmqqz259Mva9h6:3:CL:63796:LauzhackDemoRevoke3:CL_ACCUM:d641ffcc-2abf-4627-a04a-50b9e4b281f8'

# 5 revoke_credentials

In [173]:
headers = CaseInsensitiveDict()
headers["accept"] = "application/json"

res11 = rq.get(f'{host}:9020/revocation/registry/{revoc_reg_id}/issued/details', headers=headers)
res11.text

'[{"cred_def_id": "GxgDCXeCGmqqz259Mva9h6:3:CL:63796:LauzhackDemoRevoke3", "cred_ex_id": "b6e914ea-c5ff-4493-9ad4-05e223790dc4", "record_id": "2a1726ad-fbfa-4112-9863-955435fbf95e", "state": "issued", "cred_rev_id": "1", "created_at": "2022-12-03T16:54:44.497661Z", "rev_reg_id": "GxgDCXeCGmqqz259Mva9h6:4:GxgDCXeCGmqqz259Mva9h6:3:CL:63796:LauzhackDemoRevoke3:CL_ACCUM:d641ffcc-2abf-4627-a04a-50b9e4b281f8", "updated_at": "2022-12-03T16:54:44.497661Z"}, {"cred_def_id": "GxgDCXeCGmqqz259Mva9h6:3:CL:63796:LauzhackDemoRevoke3", "cred_ex_id": "6da05780-0d84-4cfa-8cc8-d0c49b6652d8", "record_id": "6eda46a9-b493-48ef-a259-3dcb1aa10832", "state": "issued", "cred_rev_id": "2", "created_at": "2022-12-03T16:56:18.110320Z", "rev_reg_id": "GxgDCXeCGmqqz259Mva9h6:4:GxgDCXeCGmqqz259Mva9h6:3:CL:63796:LauzhackDemoRevoke3:CL_ACCUM:d641ffcc-2abf-4627-a04a-50b9e4b281f8", "updated_at": "2022-12-03T16:56:18.110320Z"}]'

In [174]:
cred_ex_id = res11.json()[0]['cred_ex_id']
cred_ex_id

'b6e914ea-c5ff-4493-9ad4-05e223790dc4'

In [185]:
headers = CaseInsensitiveDict()
headers["accept"] = "application/json"
headers["Content-Type"] = "application/json"

data = """
{"connection_id":"%s",
"cred_ex_id": "%s",
"notify": true
}
"""% (res2.json()['connectionId'], cred_ex_id)

res12 = rq.post(f'{host}:9020/revocation/revoke', headers=headers, data=data)
res12.text

'{}'

In [190]:
headers = CaseInsensitiveDict()
headers["accept"] = "application/json"

res13 = rq.get(f'{host}:9020/revocation/registry/{revoc_reg_id}/tails-file', headers=headers)
res13

<Response [200]>